# Pipelines with Pandas for Data Science

In [77]:
# A Group Company has Two Branches, 
# Each Employee of the company submits their expense statement using an Input Program in their office & the Data is stored in the company data base directory as T_Exp_Data 

#Company A
T_Exp_Data_1 ={
    'Employee_Code':['001','002','003','004','005'],
    'Name':['A','B','C','D','E'],
    'Gender':['Male', 'Female', 'Male', 'Female', 'Male'],
    'Status':['Permanent', 'Probation', 'Adhoc', 'Permanent', 'Probation'],
    'Travel':['Yes','No','Yes','No','No'],
    'Food':[900,325,505,650,200],
    'Taxi':[0,20,10,5,50]
}

#Company B
T_Exp_Data_2 ={
    'Employee_Code':['101','102','103','104','105'],
    'Name':['A','B','C','D','E'],
    'Gender':['Female', 'Male', 'Female', 'Male', 'Female'],
    'Status':['Adhoc', 'Probation', 'Permanent', 'Adhoc', 'Probation'],
    'Travel':['Yes','Yes','No','Yes','Yes'],
    'Food':[350,325,475,600,215],
    'Taxi':[0,20,10,5,50]
}

print("Data Ok")

Data Ok


In [73]:
# Now we need to get this data from the database to the expense calculator programme
# We would perform calculations with Python using pandas
# 1. Set the Pandas Environment
import pandas as pd
from pandas import DataFrame
print('Environment Set')

Environment Set


In [89]:
# 2.  Government gets Data From Company A & B and Create a DataFrame for Calculation and create a new dataframe Where 'Travel' = 'Yes'

dfA=pd.DataFrame(T_Exp_Data_1) # Create Dataframe of Data from Company 1
dfB=pd.DataFrame(T_Exp_Data_2) # Create Dataframe of Data from Company 2
dfn = pd.concat([dfA,dfB])     # Combine all the Data
dfT=dfn[dfn.Travel == 'Yes']   # Clean Data, Keep only Travel Data, Since tax Refund is only on Travel Expenses
dfT.set_index('Employee_Code', inplace=True)
dfT

,Name,Gender,Status,Travel,Food,Taxi
Employee_Code,,,,,,
001,A,Male,Permanent,Yes,900,0
003,C,Male,Adhoc,Yes,505,10
101,A,Female,Adhoc,Yes,350,0
102,B,Male,Probation,Yes,325,20
104,D,Male,Adhoc,Yes,600,5
105,E,Female,Probation,Yes,215,50


In [16]:
pip install pdpipe

     |████████████████████████████████| 51kB 6.9MB/s eta 0:00:011
     |████████████████████████████████| 61kB 15.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6c/1b/9c/71fa8b5df01cc605c4b5cadcd81df76d520149ca5c02b4013b
Successfully built decore
Note: you may need to restart the kernel to use updated packages.


In [36]:
import pdpipe as pdp
print("Import Successful")

Import Successful


In [151]:
Pipeline1 = pdp.ColDrop('Travel')
Pipeline1+= pdp.ColDrop('Name')

dfD = Pipeline1(dfT)
dfD

,Gender,Status,Food,Taxi
Employee_Code,,,,
001,Male,Permanent,900,0
003,Male,Adhoc,505,10
101,Female,Adhoc,350,0
102,Male,Probation,325,20
104,Male,Adhoc,600,5
105,Female,Probation,215,50


In [104]:
Pipeline2 = pdp.ColDrop('Status')
dfD2=Pipeline2(dfD)
dfD2
Pipeline3= pdp.ColDrop('Gender')
dfD3=Pipeline3(dfD2)
dfD3

,Food,Taxi
Employee_Code,,
001,900,0
003,505,10
101,350,0
102,325,20
104,600,5
105,215,50


In [94]:
Total =dfD3.sum().sum()
#TRR = Tax_Refund_Rate, TTR = Total Tax Refund
TRR = 5/100
TTR = TRR*Total
print("Total_Tax_Refund: ", TTR , "USD")

Total_Tax_Refund:  149.0 USD


# Scenario 2
Show Pipelines can move data as well as functions for various calculations

In [119]:
# Create two functions such as 
def mean_expense_by_group(dataframe, col):
    # groups the data by a column and returns the mean food expende per group
    return dataframe.groupby(col).mean()

def uppercase_column_name(dataframe):
    # Capitalizes all the column headers
    dataframe.columns = dataframe.columns.str.upper()
    # And returns them
    return dataframe

In [123]:
# Then Create an appropriate Dataframe to be applied
# We can reuse the Dataframe which was the result of Pipeline 2 action i.e. dfD2

df = dfD2.drop(columns = ['Taxi'], axis =1)
df.set_index('Gender', inplace = True)
df

,Food
Gender,
Male,900
Male,505
Female,350
Male,325
Male,600
Female,215


In [124]:

# Now Pipeline the function over the target dataframe

A = df.pipe(mean_expense_by_group, col='Gender')
A

,Food
Gender,
Female,282.5
Male,582.5


In [122]:

# Now pipe-in two functions together to action on the target dataframe

B = (df.pipe(mean_expense_by_group, col='Gender'). pipe(uppercase_column_name))
B

,FOOD
Gender,
Female,282.5
Male,582.5


In [145]:
#or pipe-in functions one by one to action on the target dataframe

B = (A.pipe(uppercase_column_name))
B

,FOOD
Gender,
Female,282.5
Male,582.5


In [147]:
# Adding complexity
# We use Pipeline 3 action of on dfD created after Pipeline 1 further fork the same data for One Hot Encoding Method
# We know that ourpipeline 3 was "Pipeline3= pdp.ColDrop('Gender')"

Pipeline3= pdp.ColDrop('Gender')
Pipeline3+= pdp.OneHotEncode('Status')
dfD4 = Pipeline3(dfD)
dfD4

,Food,Taxi,Status_Permanent,Status_Probation
Employee_Code,,,,
001,900,0,1,0
003,505,10,0,0
101,350,0,0,0
102,325,20,0,1
104,600,5,0,0
105,215,50,0,1


In [157]:
#or Multipipe Implementation
Pipeline4 = pdp.OneHotEncode('Status')
dfD5 = (dfT.pipe(Pipeline1).pipe(pdp.ColDrop('Gender')).pipe(Pipeline4).pipe(uppercase_column_name))
dfD5

,FOOD,TAXI,STATUS_PERMANENT,STATUS_PROBATION
Employee_Code,,,,
001,900,0,1,0
003,505,10,0,0
101,350,0,0,0
102,325,20,0,1
104,600,5,0,0
105,215,50,0,1
